In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

sns.set_theme(style='whitegrid')
drivers = ['plaintext', 'emp']
driver_labels = {'plaintext':'#mux', 'emp':'running time (ms)'}

def read_stats(cat, dr, v, n, **kwargs):
    df = pd.read_csv(f'{cat}/test_{v}.{dr}.output.csv', **kwargs)
    df.columns = [n]
    # Convert to milliseconds.
    if dr == 'emp': df /= 1000
    return df

In [ ]:
# Load decision tree tests.

views = {'max':'max height', 'height':'exact height', 'spine':'spine', 'spineF':'spine w/ feat.', 'all':'all'}

stats = {}
for dr in drivers:
    df = pd.concat((read_stats('dtree', dr, 'dtree_'+v, n, usecols=['stat']) for (v, n) in views.items()), axis=1)
    df.index = ['small', 'very sparse', 'eighth sparse', 'full']
    stats[dr] = df

In [ ]:
stats['plaintext']

In [ ]:
stats['emp']

In [ ]:
ax = stats['plaintext'].T.plot.bar(logy=True, ylabel=driver_labels['plaintext'], rot=0)
ax.set_ylim(top=10**6.8)
plt.savefig('figs/dtree-plaintext.pdf')

In [ ]:
stats['emp'].T.plot.bar(logy=True, ylabel=driver_labels['emp'], rot=0)
plt.savefig('figs/dtree-emp.pdf')

In [ ]:
# Keep track of correlations between #mux and the running time.
corrs = {}

In [ ]:
# Load tree tests.

tests = {'lookup':'lookup', 'prob':'prob', 'map':'map'}

stats = {}
for dr in drivers:
    stats[dr] = pd.concat((read_stats('tree', dr, v, n, index_col='public') for (v, n) in tests.items()), axis=1)

In [ ]:
stats['plaintext']

In [ ]:
stats['emp']

In [ ]:
# Remove the first value of map because it is zero which breaks logarithmic scale.
stats['plaintext'].at[0, 'map'] = math.nan
stats['plaintext'].plot.line(xlabel='max depth', ylabel=driver_labels['plaintext'], logy=True)
plt.savefig('figs/tree-plaintext.pdf')

In [ ]:
stats['emp'].plot.line(xlabel='max depth', ylabel=driver_labels['emp'], logy=True)
plt.savefig('figs/tree-emp.pdf')

In [ ]:
fig, ax = plt.subplots(ncols=len(tests), figsize=(15,5))
ax[0].set_ylabel(driver_labels['emp'])
if 'tree' not in corrs: corrs['tree'] = {}
for i, tst in enumerate(tests.values()):
    df = pd.DataFrame(data={'#mux':stats['plaintext'][tst], tst:stats['emp'][tst]})
    corrs['tree'][tst] = df.corr().iat[0, 1]
    df.plot.line(ax=ax[i], x='#mux', y=tst, xlabel=driver_labels['plaintext'], marker='o')

plt.savefig('figs/tree-mux-corr.pdf')

In [ ]:
# Load list tests

tests = {'elem':'elem', 'hamming_distance':'hamming', 'min_euclidean_distance':'euclidean', 'dot_prod':'dot-prod'}

stats = {}
for dr in drivers:
    stats[dr] = pd.concat((read_stats('list', dr, v, n, index_col='public') for (v, n) in tests.items()), axis=1)

In [ ]:
stats['plaintext']

In [ ]:
stats['emp']

In [ ]:
stats['plaintext'].plot.line(xlabel='max length', ylabel=driver_labels['plaintext'])
plt.savefig('figs/list-plaintext-1.pdf')

In [ ]:
stats['emp'].plot.line(xlabel='max length', ylabel=driver_labels['emp'])
plt.savefig('figs/list-emp-1.pdf')

In [ ]:
fig, ax = plt.subplots(ncols=len(tests), figsize=(15,5))
ax[0].set_ylabel(driver_labels['emp'])
if 'list' not in corrs: corrs['list'] = {}
for i, tst in enumerate(tests.values()):
    df = pd.DataFrame(data={'#mux':stats['plaintext'][tst], tst:stats['emp'][tst]})
    corrs['list'][tst] = df.corr().iat[0, 1]
    df.plot.line(ax=ax[i], x='#mux', y=tst, xlabel=driver_labels['plaintext'], marker='o')

plt.savefig('figs/list-mux-corr-1.pdf')

In [ ]:
tests = {'insert':'insert', 'filter':'filter', 'map':'map'}

stats = {}
for dr in drivers:
    stats[dr] = pd.concat((read_stats('list', dr, v, n, index_col='public') for (v, n) in tests.items()))

In [ ]:
stats['plaintext'].plot.line(xlabel='max length', ylabel=driver_labels['plaintext'])
plt.savefig('figs/list-plaintext-2.pdf')

In [ ]:
stats['emp'].plot.line(xlabel='max length', ylabel=driver_labels['emp'])
plt.savefig('figs/list-emp-2.pdf')

In [ ]:
fig, ax = plt.subplots(ncols=len(tests), figsize=(15,5))
ax[0].set_ylabel(driver_labels['emp'])
if 'list' not in corrs: corrs['list'] = {}
for i, tst in enumerate(tests.values()):
    df = pd.DataFrame(data={'#mux':stats['plaintext'][tst], tst:stats['emp'][tst]})
    corrs['list'][tst] = df.corr().iat[0, 1]
    df.plot.line(ax=ax[i], x='#mux', y=tst, xlabel=driver_labels['plaintext'], marker='o')

plt.savefig('figs/list-mux-corr-2.pdf')

In [ ]:
cats = ['list', 'tree']
df = pd.concat((pd.DataFrame(data=corrs[c], index=['correlation']).T for c in cats), keys=cats)
df

In [ ]:
df.style.to_latex('figs/mux-corr.tex', hrules=True, clines='skip-last;data')

In [ ]:
# Done!